In [3]:
from ktms import *
from MAInit import *

In [9]:
#############################################################################3
# name = "CoPt3_100_u2"
# name = 'Pt_100_u2_no20_CO_n1_d9'
name = 'NiGa_110_u1_no06_CO_n1_d3'
# name = 'Pt_100_u2_no20_CO_n1_d9'
# name = 'Cu_100_u2_no20_CO_n1_d8'

inname = name + '.traj'
atoms = init_query(inname, 'spacom')
bareatoms, poslis = removemolecule(atoms, ['C', 'O'])

In [10]:
barestruct = AseAtomsAdaptor.get_structure(bareatoms)
baresites = getadsites(bareatoms, False)
d = 1.13

maxmole = 5
mindist = 1.8 # must be not 0
molecule = Atoms('CO', positions=[(0., 0., 0.), (0., 0., d)], cell=(10., 10., 10.))

In [11]:
sites0_ = baresites['all']
sites0 = [list(i) for i in sites0_]

group = creategroup(bareatoms, sites0)

rused = []
for i in reversed(range(len(sites0))):
    for j in range(len(poslis)):
        if np.allclose(sites0[i][:2], poslis[j][:2]):
            tmp = sites0.pop(i)
            rused.append(tmp)
baresites, rused

Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!
Symmetrically same structure found!


({'ontop': [array([ 0.        ,  0.        , 18.24061245]),
   array([ 1.47092646,  2.08020415, 18.24061245])],
  'bridge': [array([1.47092646e+00, 9.23796217e-16, 1.82406124e+01]),
   array([ 0.73546323,  1.04010207, 18.24061245]),
   array([ 2.20638969,  1.04010207, 18.24061245]),
   array([1.30644514e-15, 2.08020415e+00, 1.82406124e+01]),
   array([ 0.73546323,  3.12030622, 18.24061245]),
   array([ 2.20638969,  3.12030622, 18.24061245])],
  'hollow': [array([ 1.47092646,  0.69340138, 18.24061245]),
   array([1.30644514e-15, 1.38680277e+00, 1.82406124e+01]),
   array([ 2.94185292,  2.77360553, 18.24061245]),
   array([ 1.47092646,  3.46700691, 18.24061245])],
  'all': [array([ 0.        ,  0.        , 18.24061245]),
   array([ 1.47092646,  2.08020415, 18.24061245]),
   array([ 0.73546323,  1.04010207, 18.24061245]),
   array([ 2.94185292,  2.08020415, 18.24061245]),
   array([ 2.20638969,  1.04010207, 18.24061245]),
   array([ 1.47092646,  0.        , 18.24061245]),
   array([ 0.735

In [12]:
group

[[[0.0, 0.0, 18.24061244598053]],
 [[1.4709264597699998, 2.0802041486601768, 18.24061244598053]],
 [[0.7354632298849973, 1.0401020743300886, 18.24061244598053],
  [2.206389689655, 1.0401020743300886, 18.24061244598053],
  [0.735463229885, 3.1203062229902656, 18.24061244598053],
  [2.206389689655, 3.1203062229902656, 18.24061244598053]],
 [[2.9418529195399983, 2.080204148660177, 18.24061244598053]],
 [[1.4709264597699998, 0.0, 18.24061244598053]],
 [[2.9418529195399983, 1.3868027657734514, 18.24061244598053],
  [2.9418529195399987, 2.773605531546903, 18.24061244598053]],
 [[1.4709264597699998, 0.6934013828867257, 18.24061244598053],
  [1.4709264597700014, 3.467006914433629, 18.24061244598053]]]

In [6]:
allatoms, allused, mindistlis, numdict, molenum \
    = getuniqueatoms(atoms, bareatoms, sites0, maxmole, mindist, rused, group, molecule)

Used initialized!
1-------Pt16CO-------
2-------Pt16COCO-------
3-------Pt16COCOCO-------
4-------Pt16COCOCOCO-------
Distance 1.41 is below 1.8
5-------Pt16COCOCOCOCO-------
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
4-------Pt16COCOCOCO-------
Symmetrically same structure found!
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
5-------Pt16COCOCOCOCO-------
Distance 1.41 is below 1.8
5-------Pt16COCOCOCOCO-------
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
3-------Pt16COCOCO-------
Symmetrically same structure found!
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
4-------Pt16COCOCOCO-------
Symmetrically same structure found!
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
5-------Pt16COCOCOCOCO-------
5-------Pt16COCOCOCOCO-------
Distance 1.41 is below 1.8
4-------Pt16COCOCOCO-------
Symmetrically same structure

Symmetrically same structure found!
Used initialized!
1-------Pt16CO-------
2-------Pt16COCO-------
3-------Pt16COCOCO-------
4-------Pt16COCOCOCO-------
2-------Pt16COCO-------
Symmetrically same structure found!


In [7]:
print(len(allatoms))

for num in numdict:
    print(num, numdict[num])

68
1 3
2 11
3 17
4 24
5 13


In [8]:
mindistlis

[5.643844235708376,
 2.8219221178541876,
 2.8219221178541876,
 2.8219221178541876,
 1.9954002655149954,
 1.9954002655149954,
 1.9954002655149952,
 1.9954002655149954,
 1.995400265514996,
 1.9954002655149952,
 1.9954002655149952,
 1.9954002655149952,
 1.995400265514996,
 1.995400265514996,
 1.995400265514996,
 2.8219221178541876,
 2.8219221178541822,
 3.9908005310299997,
 1.995400265514996,
 1.9954002655149952,
 1.9954002655149952,
 1.995400265514996,
 3.1550048413660683,
 1.9954002655150052,
 1.9954002655150003,
 1.9954002655149985,
 1.995400265514999,
 2.8219221178541867,
 1.9954002655149952,
 1.9954002655149952,
 1.995400265514996,
 1.9954002655149952,
 1.9954002655149952,
 1.9954002655149952,
 1.995400265514996,
 5.643844235708375,
 1.9954002655150032,
 1.9954002655149998,
 1.9954002655149998,
 1.9954002655149998,
 1.9954002655149998,
 1.9954002655149998,
 1.9954002655149952,
 1.9954002655149954,
 1.9954002655149954,
 1.9954002655149954,
 1.9954002655149998,
 1.9954002655149954,
 1.

In [10]:
count = 0
indexlist = []
for i in range(len(mindistlis)):
    if mindistlis[i] > 2.5:
        indexlist.append(i)
        count += 1
# count, indexlist

In [11]:
for i in range(len(allatoms)): # len(allatoms)
    if 'COCOCOCOCO' in str(allatoms[i].symbols):
        view(allatoms[i])
        print(allatoms[i].symbols)
#         print('a')

Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO


In [30]:
for i in range(len(allatoms)):
    if 'COCOCOCOCO' in str(allatoms[i].symbols):
        outname = name + str('_no') + str('{0:03d}'.format(i+1)) +'_CO_n' + str(molenum[i]) + str('_d') + str(int(np.ceil(mindistlis[i]/0.5)-3))  + '.traj'
        print(outname)
        outpath = '/home/katsuyut/init/' + str(outname)
#         allatoms[i].write(outpath)

Pt_100_u2_no005_CO_n5_d1.traj
Pt_100_u2_no007_CO_n5_d1.traj
Pt_100_u2_no008_CO_n5_d1.traj
Pt_100_u2_no011_CO_n5_d1.traj
Pt_100_u2_no012_CO_n5_d1.traj
Pt_100_u2_no021_CO_n5_d1.traj
Pt_100_u2_no026_CO_n5_d1.traj
Pt_100_u2_no034_CO_n5_d1.traj
Pt_100_u2_no040_CO_n5_d1.traj
Pt_100_u2_no041_CO_n5_d1.traj
Pt_100_u2_no043_CO_n5_d1.traj
Pt_100_u2_no044_CO_n5_d1.traj
Pt_100_u2_no045_CO_n5_d1.traj
